# 多分类和多输出
[多分类](https://www.bilibili.com/video/BV1qf4y1x7kB?p=19)

 .| Number of targets输出目标，维度|Target cardinality目标基数，输出类型|type_of_target
 ---|---|---|---
Multiclass classification             | 1  | >2         | ‘multiclass’
Multilabel classification             | >1 | 2 (0 or 1) | ‘multilabel-indicator’
Multiclass-multioutput classification | >1 | >2         | ‘multiclass-multioutput’
Multioutput regression                | >1 | Continuous | ‘continuous-multioutput’

sklearn.utils.multiclass.type_of_target
- 'continuous'：y是浮点数的数组，不是全部整数，而是1d或列向量。
- 'continuous-multioutput'：y是一个二维数组，不是全部都是整数，并且两个维的大小都大于1。
- 'binary'：y包含<= 2个离散值，为1d或列向量。
- 'multiclass'：y包含两个以上的离散值，不是序列序列，并且是1d或列向量。
- 'multiclass-multioutput'：y是一个二维数组，包含两个以上的离散值，不是序列序列，且两个维度的大小均大于1。
- “ multilabel-indicator”：y是一个标签指示符矩阵，是一个二维数组，至少包含两列，并且最多包含2个唯一值。
- “unknown”：y类似数组，但以上都不是，例如3d数组，序列序列或非序列对象的数组。
```
>>> import numpy as np
>>> type_of_target([0.1, 0.6])
'continuous'
>>> type_of_target([1, -1, -1, 1])
'binary'
>>> type_of_target(['a', 'b', 'a'])
'binary'
>>> type_of_target([1.0, 2.0])
'binary'
>>> type_of_target([1, 0, 2])
'multiclass'
>>> type_of_target([1.0, 0.0, 3.0])
'multiclass'
>>> type_of_target(['a', 'b', 'c'])
'multiclass'
>>> type_of_target(np.array([[1, 2], [3, 1]]))
'multiclass-multioutput'
>>> type_of_target([[1, 2]])
'multilabel-indicator'
>>> type_of_target(np.array([[1.5, 2.0], [3.0, 1.6]]))
'continuous-multioutput'
>>> type_of_target(np.array([[0, 1], [1, 1]]))
'multilabel-indicator'
```

## Multiclass 单标签多分类

基本上所有的分类模型都支持多分类，只是分类策略不同，分类策略包含两种(实际上有多种，sklearn暴露的有3种通用的分类策略)：
- one-vs-the-rest / one-vs-all （multiclass.OneVsRestClassifier）
在one-vs-all策略中，假设有n个类别，那么就会建立n个二项分类器，每个分类器针对其中一个类别和剩余类别进行分类。
进行预测时，利用这n个二项分类器进行分类，得到数据属于当前类的概率，选择其中概率最大的一个类别作为最终的预测结果。
> 可解释性好。

![](img/one-vs-the-rest.jpg)
![](img/one-vs-the-rest.png)
- one-vs-one （multiclass.OneVsOneClassifier）
在one-vs-one策略中，同样假设有n个类别，则会针对两两类别建立二项分类器，得到k=n*(n-1)/2个分类器（C(N,2)种不同的分类器）。
对新数据进行分类时，依次使用这k个分类器进行分类，每次分类相当于一次投票，分类结果是哪个就相当于对哪个类投了一票。
在使用全部k个分类器进行分类后，相当于进行了k次投票，选择得票最多的那个类作为最终分类结果​。
(每个二项分类器需要学习一部分数据，而不是全部数据。可能对不能处理大规模数据的核函数算法有利)
![](img/one-vs-one.png)
> 耗时较多(O（n ^ 2）复杂性)，但较为准确。

无论是multiclass，还是multilabel，做分类时都有两种策略，一个是one-vs-​the-rest(one-vs-all)，一个是one-vs-one。

OvO和OvR的区别
![](img/one_vs_rest.png)

- error correcting output codes （multiclass.OutputCodeClassifier）
将模型构建应用分为两个阶段：编码阶段和解码阶段；编码阶段中对K个类别中进行M次划分，每次划分将一部分数据分 为
    正类，一部分数据分为反类，每次划分都构建出来一个模型， 模型的结果是在空间中对于每个类别都定义了一个点；解码阶段
    中使用训练出来的模型对测试样例进行预测，将预测样本对应的 点和类别之间的点求距离，选择距离最近的类别作为最终的预
    测 类别。
![](img/Error_Correcting.png)

## mutillabel 多标签
训练数据的类别标签Y应该是一个矩阵，第[i,j]个元素指明了第j个类别标签是否出现在第i个样本数据中。sklearn.preprocessing.MultiLabelBinarizer提供了这个功能
> 特别注意，在多标签的情况下，输入必须是二值化的。所以需要MultiLabelBinarizer()先处理。

支持multilabel的分类器:
- tree.DecisionTreeClassifier
- tree.ExtraTreeClassifier
- ensemble.ExtraTreesClassifier
- neighbors.KNeighborsClassifier
- neural_network.MLPClassifier
- neighbors.RadiusNeighborsClassifier
- ensemble.RandomForestClassifier
- linear_model.RidgeClassifierCV

## multioutput 多标签多分类

支持multiclass-multioutput:

- tree.DecisionTreeClassifier
- tree.ExtraTreeClassifier
- ensemble.ExtraTreesClassifier
- neighbors.KNeighborsClassifier
- neighbors.RadiusNeighborsClassifier
- ensemble.RandomForestClassifier

### multioutput.MultiOutputClassifier
该策略包括为每个目标配备一个分类器。这是扩展不原生支持多目标分类的分类器的简单策略

In [2]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# 这个方法相当于做了MultiLabelBinarizer()
X, y = make_multilabel_classification(n_classes=3, random_state=0)
clf = MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)
clf.predict(X[-2:]) # array([[1, 1, 0], [1, 1, 1]])

array([[1, 1, 0],
       [1, 1, 1]])

In [3]:
clf2 = KNeighborsClassifier().fit(X, y)
clf2.predict(X[-2:]) # array([[1, 1, 0], [1, 1, 1]])

array([[1, 1, 0],
       [1, 1, 1]])

In [4]:
clf3 = MultiOutputClassifier(SVC()).fit(X, y)
clf3.predict(X[-2:]) # array([[1, 1, 1], [1, 0, 1]])

array([[1, 1, 1],
       [1, 0, 1]])

In [5]:
clf3 = SVC().fit(X, y) # 报错，不支持multioutput

ValueError: y should be a 1d array, got an array of shape (100, 3) instead.

In [6]:
# demo2
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np
X, y1 = make_classification(n_samples=10, n_features=100, n_informative=30, n_classes=3, random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape # 10,100
n_outputs = Y.shape[1] # 3
n_classes = 3
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X, Y).predict(X)
# 这里输出的就不是01数组了

array([[2, 2, 0],
       [1, 2, 1],
       [2, 1, 0],
       [0, 0, 2],
       [0, 2, 1],
       [0, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 2],
       [2, 0, 0]])

### multioutput.ClassifierChain
https://scikit-learn.org/stable/modules/multiclass.html#classifierchain

y标签之间相互依赖（链式）

黄色部分是输入空间，白色部分代表目标变量。
![](img/ClassifierChain.png)

### multioutput.MultiOutputRegressor
### multioutput.RegressorChain